In [48]:
from langgraph.graph import END, StateGraph, add_messages
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from dotenv import load_dotenv
from typing import TypedDict, Annotated, List
from langgraph.types import Command, interrupt
from langchain_openai import ChatOpenAI
from langchain_community.tools import TavilySearchResults
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver
load_dotenv()

True

In [49]:
llm = ChatOpenAI(model="gpt-4o", temperature=0.0)

In [50]:
class State(TypedDict):
    topic : str
    generated_post : Annotated[List[str], add_messages]
    human_feedback : Annotated[List[str], add_messages]
    count : int

In [51]:
def generate_post_with_feedback(state:State) -> State:
    """
    Generate a social media post based on the given topic and gather human feedback.
    """
    print("post generation started")
    topic = state["topic"]
    # print("topic received", topic)
    # Simulate generating a post
    feedback = state["human_feedback"] if "human_feedback" in state else ["No feedback yet."]
    # print("feedback received", feedback)

    prompt = f"""
    Linkedin Topic : {topic}
    feedback :{feedback[-1] if feedback else "No feedback yet."}

    Generate a structured linkedin post based on above topic and feedback.
    consider the feedback to improve the post.
    """

    print("prompt for post generation", prompt)
    response = llm.invoke([
        SystemMessage(content="You are an expert linkedin media post generator."),
        HumanMessage(content=prompt)
    ])

    generated_post = response.content

    print("post generation completed", generated_post)

    return {
        "generated_post": [AIMessage(content=generated_post)],
        "human_feedback": feedback
    }
    # return None

In [ ]:
def feedback_node(state: State) -> State:
    """
    Simulate human feedback on the generated post.
    """
    # In a real application, this would be replaced with actual human input


    # user_feedback = interrupt({
    #     "generated_post":state["generated_post"],
    #     "message":"Please provide your feedback or say done to finish.",
    # })

    if state['count'] == 0:
        user_feedback = "This is a great post, but it could be shorter."
        state['count'] += 1
    else:
        user_feedback ="done"

    state["human_feedback"].append(HumanMessage(content=user_feedback))

    return state

In [53]:
def feedback_router(state: State) -> str:
    """
    Route to the appropriate node based on user feedback.
    """
    if state["human_feedback"] and state["human_feedback"][-1].content.lower() == "done":
        return "end_node"
    else:
        return "generate_post_with_feedback"

In [54]:
def end_node(state: State) -> State:
    """
    End the conversation and return the final post.
    """
    final_post = state["generated_post"][-1].content
    print(f"Final Post: {final_post}")
    return END

In [55]:
flow = StateGraph(State)
flow.add_node("generate_post_with_feedback",generate_post_with_feedback)
flow.add_node("feedback_node",feedback_node)
flow.add_node("end_node",end_node)
flow.set_entry_point("generate_post_with_feedback")
flow.add_edge("generate_post_with_feedback", "feedback_node")
flow.add_conditional_edges("feedback_node",feedback_router)
flow.add_edge("feedback_node",END)
flow.add_edge("generate_post_with_feedback",END)
# flow.add_conditional_edges(
#     "feedback_node",
#     {
#         "end_node": lambda state: state["feedback_node"][-1].content.lower() == "done",
#         "generate_post_with_feedback": lambda state: state["feedback_node"][-1].content.lower() != "done"
#     }
# )
checkpointer = MemorySaver()
app = flow.compile(checkpointer=checkpointer)
config = {
    "configurable":{
        "thread_id":1
    }
}

In [56]:
topic = input("Enter the topic for the LinkedIn post: ")

initial_state = {
    "topic": topic,
    "generated_post": [],
    "human_feedback": [],
    "count" : 0
}

result = app.invoke(initial_state,config=config)

post generation started
prompt for post generation 
    Linkedin Topic : AI killing jobs
    feedback :No feedback yet.

    Generate a structured linkedin post based on above topic and feedback.
    consider the feedback to improve the post.
    
post generation completed **Title: Navigating the AI Revolution: Embracing Change and Creating Opportunities**

In recent years, the rapid advancement of Artificial Intelligence (AI) has sparked a heated debate about its impact on the job market. While it's true that AI is transforming industries and automating tasks, it's crucial to shift our perspective from fear to opportunity.

🔍 **Understanding the Shift:**
AI is not just about replacing jobs; it's about evolving them. Routine and repetitive tasks are being automated, allowing us to focus on more strategic, creative, and human-centric roles. This shift demands a proactive approach to upskilling and reskilling.

🌟 **Opportunities for Growth:**
1. **New Job Creation:** AI is generating new

GraphRecursionError: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT

In [ ]:
# for event in result:
#     for node_id, value in event.items():
#         print(f"Node: {node_id}, Value: {value}")
#         if node_id == "__interrupt__":
#             while True: 
#                 user_feedback = input("Provide feedback (or type 'done' when finished): ")

#                 # Resume the graph execution with the user's feedback
#                 app.invoke(Command(resume=user_feedback), config=config)

#                 # Exit loop if user says done
#                 if user_feedback.lower() == "done":
#                     break

post generation started
topic received AI
feedback received []
prompt for post generation 
    Linkedin Topic : AI
    feedback :No feedback yet.

    Generate a structured linkedin post based on above topic and feedback.
    consider the feedback to improve the post.
    
post generation completed 🌟 Embracing the Future with AI 🌟

Hello LinkedIn community! 👋

As we stand on the brink of a technological revolution, Artificial Intelligence (AI) is not just a buzzword—it's a transformative force reshaping industries, enhancing efficiencies, and unlocking new possibilities. 🚀

🔍 **Why AI Matters:**
AI is revolutionizing how we approach problem-solving, from automating mundane tasks to providing deep insights through data analysis. It's enabling businesses to innovate faster, improve customer experiences, and make more informed decisions.

💡 **Key Benefits of AI:**
1. **Efficiency & Automation:** Streamlining operations and reducing human error.
2. **Data-Driven Insights:** Turning vast am

KeyError: 'feedback_node'